In [3]:
import pandas as pd

df = pd.read_csv('transaction_data.csv', engine='python')

df = df.drop(['NumberOfItemsPurchased', 'CostPerItem', 'UserId', 'Country'], axis=1)
# df = df.drop(['ItemDescription', 'NumberOfItemsPurchased', 'CostPerItem', 'UserId', 'Country'], axis=1)

df = df.sort_values('TransactionId').reset_index(drop=True)

# 確保 ItemCode 至少包含4位數字
def check_stockcode(stockcode):
    digits = sum(c.isdigit() for c in stockcode)  # 計算數字的數量
    return digits >= 4

# 確保 ItemCode為string，並篩選符合條件的數據
df['ItemCode'] = df['ItemCode'].astype(str)
df = df[df['ItemCode'].apply(check_stockcode)]

# 截取 前四個字元
# df.loc[:, 'ItemCode'] = df['ItemCode'].apply(lambda x: str(x)[:4])  


df


,TransactionId,TransactionTime,ItemCode,ItemDescription
0,5900015,Mon Feb 12 04:26:00 IST 2018,477792,SET 7 BABUSHKA NESTING BOXES
1,5900015,Mon Feb 12 04:26:00 IST 2018,1787583,WHITE HANGING HEART T-LIGHT HOLDER
2,5900015,Mon Feb 12 04:26:00 IST 2018,1492113,WHITE METAL LANTERN
3,5900015,Mon Feb 12 04:26:00 IST 2018,1764609,KNITTED UNION FLAG HOT WATER BOTTLE
4,5900015,Mon Feb 12 04:26:00 IST 2018,1772526,CREAM CUPID HEARTS COAT HANGER
...,...,...,...,...
1083813,6397457,Wed Feb 20 08:50:00 IST 2019,473655,PLASTERS IN TIN STRONGMAN
1083814,6397457,Wed Feb 20 08:50:00 IST 2019,475251,CIRCUS PARADE LUNCH BOX
1083815,6397457,Wed Feb 20 08:50:00 IST 2019,474873,PACK OF 20 SPACEBOY NAPKINS
1083816,6397457,Wed Feb 20 08:50:00 IST 2019,475209,SPACEBOY LUNCH BOX


In [11]:


transaction = df.groupby('TransactionId')['ItemCode'].apply(list).reset_index()

# 只保留至少購買 2 個不同商品的交易
transaction['ItemCode'] = transaction['ItemCode'].apply(lambda x: list(set(x)))  # 去重複
transaction = transaction[transaction['ItemCode'].apply(len) > 1].reset_index(drop=True)

transaction


,TransactionId,ItemCode
0,5900015,"[477792, 1764609, 456330, 1772526, 1492113, 17..."
1,5900026,"[475272, 475293]"
2,5900037,"[477729, 456834, 457317, 475083, 477708, 47764..."
3,5900048,"[481152, 481194, 482160, 481173]"
4,5900070,"[480900, 441735, 457611, 472332, 460173, 45620..."
...,...,...
19934,6397413,"[435225, 1785798]"
19935,6397424,"[437472, 1785798]"
19936,6397435,"[484764, 486045, 489888, 472101, 1783845, 1783..."
19937,6397446,"[463281, 434385, 445557, 488775]"


In [12]:
import csv

with open("data_to_transaction.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(transaction['ItemCode'])

print(f"number of transaction: {len(transaction)}")


number of transaction: 19939


In [ ]:
import pandas as pd

df = pd.read_csv('transaction_data.csv', engine='python')

# 嘗試將 ItemCode 轉為 int，若失敗則移除該行
df = df[pd.to_numeric(df['ItemCode'], errors='coerce').notna()]
df['ItemCode'] = df['ItemCode'].astype(int)

# 移除 ItemDescription 為 NaN 的行
df = df.dropna(subset=['ItemDescription'])

# ItemCode → ItemDescription 
item_lookup = df[['ItemCode', 'ItemDescription']].drop_duplicates().set_index('ItemCode')['ItemDescription'].to_dict()


def find_item_description(item_code):
    return item_lookup.get(int(item_code), "Unknown")

item_lookup


{465549: 'FAMILY ALBUM WHITE PICTURE FRAME',
 482370: 'LONDON BUS COFFEE MUG',
 490728: 'SET 12 COLOUR PENCILS DOLLY GIRL ',
 459186: 'UNION JACK FLAG LUGGAGE TAG',
 1733592: 'WASHROOM METAL SIGN',
 1787247: 'check?',
 471576: 'NATURAL SLATE CHALKBOARD LARGE ',
 447867: 'SKULLS WRITING SET ',
 1783845: 'MULTI COLOUR SILVER T-LIGHT HOLDER',
 494802: 'SET OF 6 RIBBONS PERFECTLY PRETTY  ',
 460215: 'John Lewis',
 459669: 'WOODEN BOX OF DOMINOES',
 486276: 'SET OF 5 MINI GROCERY MAGNETS',
 435225: 'LUNCH BAG RED SPOTTY',
 455868: 'FOLDING UMBRELLA CREAM POLKADOT',
 477246: 'ALARM CLOCK BAKELIKE GREEN',
 477036: 'CARD CIRCUS PARADE',
 489594: 'VINTAGE CHRISTMAS TABLECLOTH',
 1771938: 'SET OF 3 HEART COOKIE CUTTERS',
 461622: 'PACK OF 12 SUKI TISSUES ',
 476028: 'PINK BABY BUNTING',
 434028: 'DISCO BALL CHRISTMAS DECORATION',
 474726: 'check',
 490329: 'ROLL WRAP VINTAGE CHRISTMAS',
 443436: 'OWL DOORSTOP',
 464751: 'FOOD CONTAINER SET 3 LOVE HEART ',
 452844: 'PINK  HEART SHAPE LOVE BUCKET 

In [17]:

sample_codes = [477792,1764609,456330,1772526,1492113,1787583]  # int
descriptions = {code: find_item_description(code) for code in sample_codes}


for code, desc in descriptions.items():
    print(f"ItemCode: {code} -> Description: {desc}")

ItemCode: 477792 -> Description: SET 7 BABUSHKA NESTING BOXES
ItemCode: 1764609 -> Description: RED WOOLLY HOTTIE WHITE HEART.
ItemCode: 456330 -> Description: GLASS STAR FROSTED T-LIGHT HOLDER
ItemCode: 1772526 -> Description: incorrectly made-thrown away.
ItemCode: 1492113 -> Description: WHITE MOROCCAN METAL LANTERN
ItemCode: 1787583 -> Description: wrongly marked carton 22804


In [13]:
import pandas as pd

df_lookup = pd.read_csv('transaction_data.csv', engine='python')

# 建立 ItemCode 與 ItemDescription 的對照表
item_lookup = df_lookup[['ItemCode', 'ItemDescription']].drop_duplicates().set_index('ItemCode')['ItemDescription'].to_dict()

# 定義查詢函數
def find_item_description(item_code):
    return item_lookup.get(str(item_code), "Unknown")

item_lookup


{465549: 'FAMILY ALBUM WHITE PICTURE FRAME',
 482370: 'LONDON BUS COFFEE MUG',
 490728: 'SET 12 COLOUR PENCILS DOLLY GIRL ',
 459186: 'UNION JACK FLAG LUGGAGE TAG',
 1733592: 'WASHROOM METAL SIGN',
 1787247: nan,
 471576: 'NATURAL SLATE CHALKBOARD LARGE ',
 447867: 'SKULLS WRITING SET ',
 1783845: 'MULTI COLOUR SILVER T-LIGHT HOLDER',
 494802: 'SET OF 6 RIBBONS PERFECTLY PRETTY  ',
 460215: 'John Lewis',
 459669: 'WOODEN BOX OF DOMINOES',
 486276: 'SET OF 5 MINI GROCERY MAGNETS',
 435225: 'LUNCH BAG RED SPOTTY',
 455868: 'FOLDING UMBRELLA CREAM POLKADOT',
 477246: nan,
 477036: 'CARD CIRCUS PARADE',
 489594: 'VINTAGE CHRISTMAS TABLECLOTH',
 1771938: 'SET OF 3 HEART COOKIE CUTTERS',
 461622: 'PACK OF 12 SUKI TISSUES ',
 476028: 'PINK BABY BUNTING',
 434028: 'DISCO BALL CHRISTMAS DECORATION',
 474726: nan,
 490329: 'ROLL WRAP VINTAGE CHRISTMAS',
 443436: nan,
 464751: 'FOOD CONTAINER SET 3 LOVE HEART ',
 452844: 'PINK  HEART SHAPE LOVE BUCKET ',
 485667: 'DOLLCRAFT GIRL NICOLE',
 469392:

In [14]:
sample_codes = [486654,477246,471576]  
# sample_codes = [1772526,1764609,456330,477792,1787583,1492113]  
# 不存在，應該回傳 "Unknown"
descriptions = {code: find_item_description(code) for code in sample_codes}

for code, desc in descriptions.items():
    print(f"ItemCode: {code} -> Description: {desc}")

ItemCode: 486654 -> Description: Unknown
ItemCode: 477246 -> Description: Unknown
ItemCode: 471576 -> Description: Unknown
